# Review DPS outputs

Make a mosaic of DPS outputs.

1. make a list of the DPS output paths with build_tindex.master.py
2. Identify duplicate tiles
3. Identify matching tiles; merge tindex.master with the original index tile file
4. make mosaicjson
6. View DPS results

In [1]:
import geopandas
import pandas as pd
import os
import json
import collections

def local_to_s3(url, user = 'nathanmthomas', type='public'):
    ''' A Function to convert local paths to s3 urls'''
    if type is 'public':
        replacement_str = f's3://maap-ops-workspace/shared/{user}'
    else:
        replacement_str = f's3://maap-ops-workspace/{user}'
    return url.replace(f'/projects/my-{type}-bucket', replacement_str)

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
DPS_DATA_TYPE = 'AGB' #"Topo" "Landsat" "ATL08" "AGB"
DPS_DATA_USER = 'lduncanson' #'nathanmthomas'
if True:
    print(f"Building master tile index for: {DPS_DATA_TYPE}")
    dps_month = '10'
    os.system(f"python /projects/icesat2_boreal/lib/build_tindex_master.py --type {DPS_DATA_TYPE} -m {dps_month}")
    
tindex_master_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master.csv'

Building master tile index for: AGB


In [3]:
# Build up a dataframe from the list of dps output files
tindex_master = pd.read_csv(tindex_master_fn)
tindex_master['s3'] = [local_to_s3(local_path, user='lduncanson') for local_path in tindex_master['local_path']]
print(tindex_master.head())

# Get boreal tiles
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.astype({'layer':'int'})
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.

# For some reason, doing this causes 'MosaicJSON.from_features()' to fail...(below)
if True:
    # Remove bad tiles
    boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
print(boreal_tile_index.head())

   Unnamed: 0                                         local_path  tile_num  \
0           0  /projects/my-private-bucket/dps_output/run_bor...         9   
1           1  /projects/my-private-bucket/dps_output/run_bor...       618   
2           2  /projects/my-private-bucket/dps_output/run_bor...       201   
3           3  /projects/my-private-bucket/dps_output/run_bor...       357   
4           4  /projects/my-private-bucket/dps_output/run_bor...       484   

                                                  s3  
0  /projects/my-private-bucket/dps_output/run_bor...  
1  /projects/my-private-bucket/dps_output/run_bor...  
2  /projects/my-private-bucket/dps_output/run_bor...  
3  /projects/my-private-bucket/dps_output/run_bor...  
4  /projects/my-private-bucket/dps_output/run_bor...  
   tile_num                                           geometry
0         1  POLYGON ((-2151478.000 9423304.000, -2061478.0...
1         2  POLYGON ((-2061478.000 9423304.000, -1971478.0...
2         3 

# Identify duplicate tiles

In [4]:
duplicate_tiles = [item for item, count in collections.Counter(tindex_master["tile_num"]).items() if count > 1]
print(duplicate_tiles)

[]


# Identify matching tiles

In [5]:
# Select the rows we have results for
tile_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(bad_tiles)], how='right', on='tile_num')
print(tile_matches.shape)

tile_matches_duplicates = boreal_tile_index.merge(tindex_master[tindex_master['tile_num'].isin(duplicate_tiles)], how='right', on='tile_num')
print(tile_matches_duplicates.shape)

tile_matches_geojson_string = tile_matches.to_crs("EPSG:4326").to_json()
tile_matches_geojson = json.loads(tile_matches_geojson_string)

tile_matches_duplicates_geojson_string = tile_matches_duplicates.to_crs("EPSG:4326").to_json()
tile_matches_duplicates_geojson = json.loads(tile_matches_duplicates_geojson_string)

(43, 5)
(0, 5)


## Build a MosaicJSON

In [6]:
from typing import Dict

from cogeo_mosaic.mosaic import MosaicJSON
from cogeo_mosaic.backends import MosaicBackend

def get_accessor(feature: Dict):
    """Return specific feature identifier."""
    return feature["properties"]["s3"]

In [7]:
out_mosaic_json_fn = f's3://maap-ops-workspace/shared/{DPS_DATA_USER}/DPS_tile_lists/{DPS_DATA_TYPE}_tindex_master_mosaic.json' 

mosaicdata = MosaicJSON.from_features(tile_matches_geojson.get('features'), minzoom=6, maxzoom=18, accessor=get_accessor)

with MosaicBackend(out_mosaic_json_fn, mosaic_def=mosaicdata) as mosaic:
    mosaic.write(overwrite=True)
    
# URL to give tiler is
# s3://maap-ops-workspace/shared/nathanmthomas/Landsat/landsat8_mosaic.json

## View the Results with Folium

In [8]:
from folium import Map, TileLayer, GeoJson, LayerControl, Icon, Marker, features, Figure

In [9]:
# Setup the mosaic tiling
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/"
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

# Get a basemap
tiler_basemap_gray = "http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}"
tiler_basemap_image = 'https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

# Get Vector layers
boreal_geojson = '/projects/shared-buckets/nathanmthomas/boreal.geojson' 
boreal = geopandas.read_file(boreal_geojson)
ecoboreal_geojson = '/projects/shared-buckets/nathanmthomas/Ecoregions2017_boreal_m.geojson'
ecoboreal = geopandas.read_file(ecoboreal_geojson)

# Reproject Vector Layers
p1, p2, clat, clon = [50, 70, 40, 160]
proj_str_aea = '+proj=aea +lat_1={:.2f} +lat_2={:.2f} +lat_0={:.2f} +lon_0={:.2f}'.format(p1, p2, clat, clon)
ecoboreal_aea = ecoboreal.to_crs(proj_str_aea)
# Apply a buffer
ecoboreal_aea_buf = ecoboreal_aea["geometry"].buffer(1e5)
# Go back to GCS
ecoboreal_buf = ecoboreal_aea_buf.to_crs(boreal_tile_index.crs)

# Style Vector Layers
ecoboreal_style = {'fillColor': 'orange', 'color': 'orange'}
boreal_style = {'fillColor': 'green', 'color': 'green'}
boreal_subset_style = {'fillColor': 'red', 'color': 'red'}

# Map the Layers
Map_Figure=Figure(width=2000,height=600)
#------------------
m1 = Map(
    tiles="Stamen Terrain",
    location=(60, 5),
    zoom_start=3
)
Map_Figure.add_child(m1)

#GeoJson(ecoboreal_aea_buf, name="Boreal extent from Ecoregions", style_function=lambda x:ecoboreal_style).add_to(m1)
GeoJson(boreal, name="Boreal extent", style_function=lambda x:boreal_style).add_to(m1)

boreal_tiles_style = {'fillColor': '#ff7f00', 'color': '#ff7f00'}
dps_subset_style = {'fillColor': '#377eb8', 'color': '#377eb8'}
dps_dup_style = {'fillColor': 'red', 'color': 'red'}

GeoJson(
        data=boreal_tile_index.to_crs("EPSG:4326").to_json(),
        style_function=lambda x:boreal_tiles_style,
        name="Boreal tiles",
    tooltip=features.GeoJsonTooltip(
            fields=['tile_num'],
            aliases=['Tile num:'],
    )
    ).add_to(m1)


GeoJson(
        data=tile_matches_geojson,
        style_function=lambda x:dps_subset_style,
        name="DPS subset of Boreal tiles",
        tooltip=features.GeoJsonTooltip(
            fields=['tile_num'],
            aliases=['Tile num:'],
    )
    ).add_to(m1)

if len(duplicate_tiles) > 0:
    GeoJson(
            data=tile_matches_duplicates_geojson,
            style_function=lambda x:dps_dup_style,
            name="Duplicate DPS subset of Boreal tiles"
        ).add_to(m1)

if True:
    
    basemap_gray = TileLayer(
        tiles=tiler_basemap_gray,
        opacity=1,
        name="World gray basemap",
        attr="MAAP",
        overlay=True
    )
    basemap_image = TileLayer(
        tiles=tiler_basemap_image,
        opacity=1,
        name="Image basemap",
        attr="MAAP",
        overlay=True
    )
    landsat_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Landsat_mosaic.json&rescale=0.01,0.5&bidx=6&colormap_name=viridis"
    landsat_tiles_layer = TileLayer(
        tiles=landsat_tiles,
        opacity=1,
        name="landsat covars",
        attr="MAAP",
        overlay=True
    )
    topo_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_mosaic.json&rescale=0,1&bidx=3&colormap_name=bone"
    topo_tiles_layer = TileLayer(
        tiles=topo_tiles,
        opacity=1,
        name="topo covars",
        attr="MAAP",
        overlay=True
    )
    agb_tiles = f"{tiler_mosaic}?url=s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_mosaic.json&rescale=0,10&bidx=1&colormap_name=viridis"
    agb_tiles_layer = TileLayer(
        tiles=agb_tiles,
        opacity=1,
        name="Boreal AGB",
        attr="MAAP",
        overlay=True
    )
    basemap_gray.add_to(m1)
    basemap_image.add_to(m1)
    #landsat_tiles_layer.add_to(m1)
    #topo_tiles_layer.add_to(m1)
    agb_tiles_layer.add_to(m1)

LayerControl().add_to(m1)

m1